In [3]:
import requests
from bs4 import BeautifulSoup
import re

import firebase_admin
from firebase_admin import credentials, firestore

from recipe_scrapers import scrape_me
import requests
from recipe_scrapers import scrape_html


In [4]:

path = 'supermercado-371012-firebase-adminsdk-lu3fc-48a8c3583d.json'
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate(path)
    firebase_admin.initialize_app(cred)

firebase_admin.get_app()

db = firestore.client()

In [5]:
def crawl_recipe_links(page):
    # Send the request and get the response
    response = requests.get(page)

    # Parse the response content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements with the class 'c-teaser__link'
    recipe_elements = soup.find_all(class_='ds-recipe-card__image-wrap ds-teaser-link__image-wrap ds-teaser-link__image-wrap')
    print(len(recipe_elements))

    # Print the text and href attributes of each element
    recipe_links = []
    for html in recipe_elements:
        html = str(html)
        soup = BeautifulSoup(html, 'html.parser')
        
        # find the img tag
        img_tag = soup.find('img')

        # access the src attribute
        src = img_tag['src']
        substring_index = src.index('bilder')
        cutoff_string = src[:substring_index]
        recipe_links.append(cutoff_string)
    return recipe_links


In [38]:
def write_recipe(title, time, ingredients, instructions, image, nutrients):
    doc_ref = db.collection(u'recipes').document(title)
    doc_ref.set({

        u'title': title,
        u'time': time,
        u'ingredients': ingredients,
        u'instructions': instructions,
        u'image': image,
        u'nutrients': nutrients,

})

In [7]:
units = ['Becher', 'Beet/e', 'Beutel', 'Blatt', 'Blätter', 'Bund', 'Bündel', 'cl', 'cm', 'dicke', 'dl', 'Dose', 'Dose/n', 'dünne', 'Ecke(n)', 'Eimer', 'einige', 'einige Stiele', 'EL', 'EL, gehäuft', 'EL, gestr.', 'etwas', 'evtl.', 'extra', 'Fässchen', 'Fläschchen', 'Flasche', 'Flaschen', 'g', 'Glas', 'Gläser', 'gr. Dose/n', 'gr. Flasche(n)', 'gr. Glas', 'gr. Gläser', 'gr. Kopf', 'gr. Scheibe(n)', 'gr. Stück(e)', 'große', 'großen', 'großer', 'großes', 'halbe', 'Halm(e)', 'Handvoll', 'Kästchen', 'kg', 'kl. Bund', 'kl. Dose/n', 'kl. Flasche/n', 'kl. Glas', 'kl. Gläser', 'kl. Kopf', 'kl. Scheibe(n)', 'kl. Stange(n)', 'kl. Stück(e)', 'kleine', 'kleiner', 'kleines', 'Knolle/n', 'Kopf', 'Köpfe', 'Körner', 'Kugel', 'Kugel/n', 'Kugeln', 'Liter', 'm.-große', 'm.-großer', 'm.-großes', 'mehr', 'mg', 'ml', 'Msp.', 'n. B.', 'Paar', 'Paket', 'Pck.', 'Pkt.', 'Platte/n', 'Port.', 'Prise(n)', 'Prisen', 'Prozent %', 'Riegel', 'Ring/e', 'Rippe/n', 'Rispe(n)', 'Rolle(n)', 'Schälchen', 'Scheibe/n', 'Schuss', 'Spritzer', 'Stange/n', 'Stängel', 'Staude(n)', 'Stick(s)', 'Stiel/e', 'Stiele', 'Streifen', 'Stück(e)', 'Tablette(n)', 'Tafel', 'Tafeln', 'Tasse', 'Tasse/n', 'Teil/e', 'TL', 'TL, gehäuft', 'TL, gestr.', 'Topf', 'Tropfen', 'Tube/n', 'Tüte/n', 'viel', 'wenig', 'Würfel', 'Wurzel', 'Wurzel/n', 'Zehe/n', 'Zweig/e']

In [63]:
def ingredient_dict(ingredient_list):
    scraper_ingredients = {}
    # ingredients are like: 1 EL Zucker
    for ingredient in ingredient_list:
        words = ingredient.split()
        is_found = False
        for word in words:
            if is_found:
                break
            # check weather one unit is contained in ingredient
            for unit in units:
                if word == unit:
                    index = ingredient.find(word) + len(word)
                    
                    key_ingredient = ingredient[index:]
                    value_unit = ingredient[:index]
                    scraper_ingredients[key_ingredient] = value_unit
                    is_found = True
                    break
        # no unit is found like: 'g' or 'ml'
        if not is_found:
            numbers = re.findall(r'\d+', ingredient)
            # check if there are numbers in ingredient in order to extract it like: 1 Apple -> Apple : 1
            if len(numbers)>0:
                print(f'Numbers found: {ingredient}')
                # first_num has type str
                first_num = numbers[0]
                new_ingredient = ingredient.replace(first_num+' ', '')
                value_unit = first_num
                scraper_ingredients[new_ingredient] =  value_unit
                print(f'After changes {new_ingredient}:{value_unit}')
                print('')
            # in this case there is no unit like: Apple
            else:
                key_ingredient = ingredient
                value_unit = "not available"
                scraper_ingredients[key_ingredient] =  value_unit
    return scraper_ingredients

In [64]:
def get_recipe_ingredients(url):
    html = requests.get(url).content

    scraper = scrape_html(html=html, org_url=url)

    ingredient_map = ingredient_dict(scraper.ingredients())
        
    try:
        write_recipe(scraper.title(),scraper.total_time(), ingredient_map, scraper.instructions(), scraper.image(),scraper.nutrients())
    except ValueError:
        print(ingredient_map)

# get_recipe_ingredients("https://www.chefkoch.de/rezepte/2737891426025016/Gyros-Nudelauflauf-in-Metaxasosse.html")

In [65]:
recipe_links = crawl_recipe_links('https://www.chefkoch.de/rs/s0t21,32/Hauptspeise-Vegetarisch-Rezepte.html')

for i, url in enumerate(recipe_links):
    get_recipe_ingredients(url)


59
Numbers found: 2 Schalotte(n)
After changes Schalotte(n):2

Numbers found: 2 Knoblauchzehe(n)
After changes Knoblauchzehe(n):2

Numbers found: 2 Lorbeerblätter
After changes Lorbeerblätter:2

Numbers found: 4 Ei(er), Größe M
After changes Ei(er), Größe M:4

Numbers found: 1 Ei(er)
After changes Ei(er):1

Numbers found: 1 EL, gestr. Gemüsebrühepulver
After changes EL, gestr. Gemüsebrühepulver:1

Numbers found: 2 Knoblauchzehe(n)
After changes Knoblauchzehe(n):2

Numbers found: 1 Schalotte(n) , klein geschnitten
After changes Schalotte(n) , klein geschnitten:1

Numbers found: 1 Tomate(n)
After changes Tomate(n):1

Numbers found: 5 Knoblauchzehe(n)
After changes Knoblauchzehe(n):5

Numbers found: 2 Peperoni
After changes Peperoni:2

{' Lauch (je nach Dicke 1 oder 2 Stangen)': '800 g', ' Kümmelsamen': '1 TL', ' Öl': '1 EL', ' Butter': '2 EL', 'Salz': 'not available', ' Wasser': '200 ml', ' Mehl': '1 EL', '': 'n. B. Milch , ca. 200 - 250 ml'}
Numbers found: 1 Tomate(n) , in Scheiben gesc